In [ ]:
### Create a spark table for a Bitvore dataset in this case the public signals 2020
### The dataset has gone through some preparation to be usable by the basic spark engine.
### The following preparation of the .csv needs to be done before upload
###     The text columns for title and excerpt were removed
###     All commas and double quotes were removed


%%pyspark

from pyspark.sql.types import FloatType

from pyspark.sql.functions import col, concat_ws
from pyspark.sql.functions import *

from pyspark.sql.types import TimestampType
from pyspark.sql.functions import unix_timestamp

### read the upload file into a dataframe
filewqual = "corp-signals-public-us-2020.csv"

### CHANGE THE @XXXXXX name to your storage workspace
thefile = "abfss://workspace@stgbvnews.dfs.core.windows.net/BitvoreDatasets/" + filewqual

df_raw = spark.read.options(header = 'True').csv(thefile)

df_raw.printSchema()

### Convert sentiment string to a float
### Convert strings representing date / times to spark timestamps and extract dates to new cols
df_conv = df_raw.withColumn("Sentiment", df_raw.Sentiment.cast('float'))\
                .withColumn("AvailableAt",unix_timestamp("AvailableAt", 'MM/dd/yyyy HH:mm').cast(TimestampType()))\
                .withColumn("dateAvailable", to_date(col("AvailableAt")) )\
                .withColumn("PublishedAt",unix_timestamp("PublishedAt", 'MM/dd/yyyy HH:mm').cast(TimestampType()))\
                .withColumn("datePub", to_date(col("PublishedAt")) )\
                .withColumn("LastModified",unix_timestamp("LastModified", 'MM/dd/yyyy HH:mm').cast(TimestampType()))\
                .withColumn("dateLastModified", to_date(col("LastModified")) )


df_conv.printSchema()

spark.sql("CREATE DATABASE IF NOT EXISTS bitvorenews")


### overwrite the  table
df_conv.write.mode("overwrite").saveAsTable("bitvorenews.signalspublic2020")




In [ ]:
%%spark
val sql_write_df = spark.sql("SELECT * FROM bitvorenews.signalspublic2021")

sql_write_df.write.mode("overwrite").sqlanalytics("bitvoresql.dbo.signalspublic2021", Constants.INTERNAL)